In [61]:
from clickhouse_driver import Client

client = Client(host='localhost')

In [7]:
%timeit -n5 -r5 client.execute('SELECT count(*) FROM default.views')

12.1 ms ± 4.65 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [76]:
%timeit -n5 -r5 client.execute('SELECT count(DISTINCT user_id) FROM default.views')

938 ms ± 139 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [77]:
%timeit -n5 -r5 client.execute('SELECT count(DISTINCT movie_id) FROM default.views')

1.2 s ± 645 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [78]:
%timeit -n5 -r5 client.execute('SELECT min(viewed_frame), max(viewed_frame) FROM default.views')

70.6 ms ± 6.89 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [75]:
%timeit -n5 -r5 client.execute('SELECT count(*), sum(viewed_frame), avg(viewed_frame) FROM default.views')

58.5 ms ± 7.4 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [79]:
%timeit -n1 -r5 client.execute('SELECT user_id, count(distinct movie_id) FROM default.views GROUP by user_id')

6.53 s ± 1.34 s per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [80]:
%timeit -n1 -r5 client.execute('SELECT movie_id, count(distinct user_id) FROM default.views GROUP by movie_id')

8.27 s ± 3.27 s per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [81]:
%timeit -n1 -r5 client.execute('SELECT user_id, sum(viewed_frame), min(viewed_frame), max(viewed_frame), avg(viewed_frame) FROM default.views GROUP by user_id')

4.01 s ± 493 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [82]:
%timeit -n1 -r5 client.execute('SELECT movie_id, sum(viewed_frame), min(viewed_frame), max(viewed_frame), avg(viewed_frame) FROM default.views GROUP by movie_id')

2.92 s ± 73 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [116]:
from datetime import datetime, timedelta
import random
import string

def date_between(start, end):
    start = datetime.strptime(start, '%Y-%m-%d')
    end = datetime.strptime(end, '%Y-%m-%d')

    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = random.randrange(int_delta)
    random_microseconds = random.randint(1, int_delta)
    return start + timedelta(seconds=random_second, microseconds=random_microseconds)

def generate_movie_id(length=7):
    letters_and_digits = string.digits
    rand_string = ''.join(random.sample(letters_and_digits, length))
    return 'tt'+rand_string

def data_row(x=0):
    return [
            str(random.randint(1, 1000000)),
            generate_movie_id(),
            # возъмем среднюю продолжительность фильма 120 мин.
            # будем сохранять каждые 30 сек. просмотра
            random.randint(1, (60*60*2)/30),
            date_between('2022-05-23', '2022-05-30')
        ]

values = [r for r in map(data_row, range(0, 1000))]

In [138]:
%timeit -n1 -r5 client.execute('INSERT INTO default.views VALUES', values)

89.7 ms ± 31.4 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [46]:
%%timeit -n1 -r1000
client.execute("""
    INSERT INTO default.views (user_id, movie_id, viewed_frame, event_time)
    VALUES ('782951', 'tt5824071', 45, now())
    """)

The slowest run took 16.70 times longer than the fastest. This could mean that an intermediate result is being cached.
9.52 ms ± 12.7 ms per loop (mean ± std. dev. of 1000 runs, 1 loop each)


In [ ]:
Тестирование обработки данных, поступающих в реальном времени

In [106]:
for _ in range(3):
    %timeit -n5 -r5 client.execute('SELECT count(*), sum(viewed_frame), avg(viewed_frame) FROM default.views')

139 ms ± 3.77 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)
123 ms ± 8.17 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)
124 ms ± 11.1 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)
